In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
from collections import Counter
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/youtube-new/GBvideos.csv
/kaggle/input/youtube-new/MXvideos.csv
/kaggle/input/youtube-new/KRvideos.csv
/kaggle/input/youtube-new/MX_category_id.json
/kaggle/input/youtube-new/GB_category_id.json
/kaggle/input/youtube-new/US_category_id.json
/kaggle/input/youtube-new/IN_category_id.json
/kaggle/input/youtube-new/DEvideos.csv
/kaggle/input/youtube-new/KR_category_id.json
/kaggle/input/youtube-new/RU_category_id.json
/kaggle/input/youtube-new/FRvideos.csv
/kaggle/input/youtube-new/USvideos.csv
/kaggle/input/youtube-new/INvideos.csv
/kaggle/input/youtube-new/RUvideos.csv
/kaggle/input/youtube-new/CA_category_id.json
/kaggle/input/youtube-new/DE_category_id.json
/kaggle/input/youtube-new/JP_category_id.json
/kaggle/input/youtube-new/JPvideos.csv
/kaggle/input/youtube-new/FR_category_id.json
/kaggle/input/youtube-new/CAvideos.csv


# **Loading Packages**

In [2]:
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

**Importing the data set for India**

In [3]:
f = open(r"../input/youtube-new/IN_category_id.json")
india_cate = json.load(f)
india_videos = pd.read_csv(r"../input/youtube-new/INvideos.csv")

In [4]:
india_videos.head(10)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,kzwfHumJyYc,17.14.11,Sharry Mann: Cute Munda ( Song Teaser) | Parmi...,Lokdhun Punjabi,1,2017-11-12T12:20:39.000Z,"sharry mann|""sharry mann new song""|""sharry man...",1096327,33966,798,882,https://i.ytimg.com/vi/kzwfHumJyYc/default.jpg,False,False,False,Presenting Sharry Mann latest Punjabi Song Cu...
1,zUZ1z7FwLc8,17.14.11,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",HJ NEWS,25,2017-11-13T05:43:56.000Z,"पीरियड्स के समय|""पेट पर पति करता ऐसा""|""देखकर द...",590101,735,904,0,https://i.ytimg.com/vi/zUZ1z7FwLc8/default.jpg,True,False,False,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं..."
2,10L1hZ9qa58,17.14.11,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,TFPC,24,2017-11-12T15:48:08.000Z,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,473988,2011,243,149,https://i.ytimg.com/vi/10L1hZ9qa58/default.jpg,False,False,False,Watch Stylish Star Allu Arjun @ ChaySam Weddin...
3,N1vE8iiEg64,17.14.11,Eruma Saani | Tamil vs English,Eruma Saani,23,2017-11-12T07:08:48.000Z,"Eruma Saani|""Tamil Comedy Videos""|""Films""|""Mov...",1242680,70353,1624,2684,https://i.ytimg.com/vi/N1vE8iiEg64/default.jpg,False,False,False,This video showcases the difference between pe...
4,kJzGH0PVQHQ,17.14.11,why Samantha became EMOTIONAL @ Samantha naga ...,Filmylooks,24,2017-11-13T01:14:16.000Z,"Filmylooks|""latest news""|""telugu movies""|""telu...",464015,492,293,66,https://i.ytimg.com/vi/kJzGH0PVQHQ/default.jpg,False,False,False,why Samantha became EMOTIONAL @ Samantha naga ...
5,il_pSa5l98w,17.14.11,"MCA (Middle Class Abbayi) TEASER - Nani,Sai Pa...",Dil Raju,24,2017-11-10T04:29:50.000Z,"Nenu Local|""Nenu Local Telugu Movie""|""Nani""|""S...",6106669,98612,4185,4763,https://i.ytimg.com/vi/il_pSa5l98w/default.jpg,False,False,False,Watch MCA- Middle Class Abbayi First Look Teas...
6,7MxiQ4v0EnE,17.14.11,Daang ( Full Video ) | Mankirt Aulakh | Sukh S...,Speed Records,10,2017-11-11T16:41:15.000Z,"punjabi songs|""punjabi bhangra""|""punjabi music...",5718766,127477,7134,8063,https://i.ytimg.com/vi/7MxiQ4v0EnE/default.jpg,False,False,False,Song - Daang\nSinger - Mankirt Aulakh\nFaceboo...
7,c64I9HNpiOY,17.14.11,Padmavati : Ek Dil Ek Jaan Video Song | Deepik...,T-Series,10,2017-11-11T06:14:19.000Z,"Ek Dil Ek Jaan Video Song|""'Ek Dil Ek Jaan'""|""...",10588371,132738,8812,10847,https://i.ytimg.com/vi/c64I9HNpiOY/default.jpg,False,False,False,Presenting the song 'Ek Dil Ek Jaan' from Padm...
8,KObFEYCaRx8,17.14.11,Chiranjeevi in Naga Chaitanya - Samantha Recep...,Top Telugu Media,24,2017-11-13T04:42:26.000Z,Chiranjeevi in Naga Chaitanya - Samantha Recep...,118223,520,53,23,https://i.ytimg.com/vi/KObFEYCaRx8/default.jpg,False,False,False,Chiranjeevi in Naga Chaitanya - Samantha Recep...
9,g8QsfJhFpjY,17.14.11,New bike vs Old bike - the reality,Jump Cuts,24,2017-11-12T04:30:01.000Z,"Jump cuts|""Jumpcuts""|""Tamil comedy""|""Tamil Com...",969030,59798,1545,2404,https://i.ytimg.com/vi/g8QsfJhFpjY/default.jpg,False,False,False,Jump Cuts is a Tamil entertaining group by Har...


# Top ten trending channels

In [5]:
def top_trending_channels(data):
    trending_channels = data.groupby(['channel_title']).sum().sort_values('views',ascending = False).head(10)
    fig1 = px.bar(trending_channels,x = trending_channels.index,y = 'views',title = 'Top Trending Channels',width = 800, height = 400)
    fig1.show()

In [6]:
top_trending_channels(india_videos)

In [7]:
def channels_with_more_trends(data):
    s = data.value_counts('channel_title').head(10)
    tot_views = data.groupby(['channel_title']).sum()
    views_vs_channel = {}
    for j in s.index.values:
        for i in range(0,len(tot_views)):
            if tot_views.index[i] == j:
                views_vs_channel[j] = tot_views.iloc[i]['views']
    rows = views_vs_channel.keys()
    columns = views_vs_channel.values()
    data = {'channel':rows,'views':columns}
    views_vs_channel_df = pd.DataFrame(data)
    fig2 = px.bar(views_vs_channel_df.sort_values('views',ascending = False),x = 'channel', y = 'views',title = 'Views of Channels with most number of trends',height = 800,width = 400)
    fig2.show()

In [8]:
channels_with_more_trends(india_videos)

# Top channels by category

In [9]:
#Adding the category data from the json file to the videos_csv file.
def getList(dict):
    return [*dict]
def add_category_data(data):
    holder = []
    for key in india_cate.items():
        holder.append(key)
    id_title_dict = {}
    for x  in holder[2]:
        for i in x[5:]:
            id_title_dict[int(i['id'])] = i['snippet']['title']
    category = []
    ids = getList(id_title_dict.keys())
    cat_names = getList(id_title_dict.values())
    for i in range(0,len(india_videos)):
        obs_id = india_videos.iloc[i]['category_id'] 
        if obs_id in ids:
            category.append(id_title_dict[obs_id])
        else:
            category.append('Unknown')
    videos_copy = data.copy(deep = True)
    videos_copy['Category'] = category
    videos_copy[videos_copy['Category'] == 'Unknown'].shape
    return videos_copy,cat_names

In [10]:
videos_with_cat,cat_names = add_category_data(india_videos)

In [11]:
#Removing duplicate categories
n_cat_names = set(cat_names)
cat_names = list(n_cat_names)

In [12]:
cat_names

['Thriller',
 'Sci-Fi/Fantasy',
 'Shorts',
 'Travel & Events',
 'Drama',
 'Movies',
 'Action/Adventure',
 'Entertainment',
 'Education',
 'Comedy',
 'Family',
 'People & Blogs',
 'Foreign',
 'Trailers',
 'Classics',
 'Anime/Animation',
 'Horror',
 'Shows',
 'Documentary',
 'Howto & Style',
 'Videoblogging',
 'News & Politics',
 'Science & Technology',
 'Short Movies',
 'Gaming']

In [13]:
videos_with_cat['Category']

0                Unknown
1        News & Politics
2          Entertainment
3                 Comedy
4          Entertainment
              ...       
37347             Comedy
37348      Entertainment
37349      Entertainment
37350      Entertainment
37351      Entertainment
Name: Category, Length: 37352, dtype: object

In [14]:
videos_with_cat

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Category
0,kzwfHumJyYc,17.14.11,Sharry Mann: Cute Munda ( Song Teaser) | Parmi...,Lokdhun Punjabi,1,2017-11-12T12:20:39.000Z,"sharry mann|""sharry mann new song""|""sharry man...",1096327,33966,798,882,https://i.ytimg.com/vi/kzwfHumJyYc/default.jpg,False,False,False,Presenting Sharry Mann latest Punjabi Song Cu...,Unknown
1,zUZ1z7FwLc8,17.14.11,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",HJ NEWS,25,2017-11-13T05:43:56.000Z,"पीरियड्स के समय|""पेट पर पति करता ऐसा""|""देखकर द...",590101,735,904,0,https://i.ytimg.com/vi/zUZ1z7FwLc8/default.jpg,True,False,False,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",News & Politics
2,10L1hZ9qa58,17.14.11,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,TFPC,24,2017-11-12T15:48:08.000Z,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,473988,2011,243,149,https://i.ytimg.com/vi/10L1hZ9qa58/default.jpg,False,False,False,Watch Stylish Star Allu Arjun @ ChaySam Weddin...,Entertainment
3,N1vE8iiEg64,17.14.11,Eruma Saani | Tamil vs English,Eruma Saani,23,2017-11-12T07:08:48.000Z,"Eruma Saani|""Tamil Comedy Videos""|""Films""|""Mov...",1242680,70353,1624,2684,https://i.ytimg.com/vi/N1vE8iiEg64/default.jpg,False,False,False,This video showcases the difference between pe...,Comedy
4,kJzGH0PVQHQ,17.14.11,why Samantha became EMOTIONAL @ Samantha naga ...,Filmylooks,24,2017-11-13T01:14:16.000Z,"Filmylooks|""latest news""|""telugu movies""|""telu...",464015,492,293,66,https://i.ytimg.com/vi/kJzGH0PVQHQ/default.jpg,False,False,False,why Samantha became EMOTIONAL @ Samantha naga ...,Entertainment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37347,iNHecA3PJCo,18.14.06,फेकू आशिक़ - राजस्थान की सबसे शानदार कॉमेडी | ...,RDC Rajasthani,23,2018-06-13T08:01:11.000Z,"twinkle vaishnav comedy|""twinkle vaishnav""|""tw...",214378,3291,404,196,https://i.ytimg.com/vi/iNHecA3PJCo/default.jpg,False,False,False,PRG Music & RDC Rajasthani presents फेकू आशिक़...,Comedy
37348,dpPmPbhcslM,18.14.06,Seetha | Flowers | Ep# 364,Flowers TV,24,2018-06-13T11:30:04.000Z,"flowers serials|""actress""|""malayalam serials""|...",406828,1726,478,1428,https://i.ytimg.com/vi/dpPmPbhcslM/default.jpg,False,False,False,"Flowers - A R Rahman Show,Book your Tickets He...",Entertainment
37349,mV6aztP58f8,18.14.06,Bhramanam I Episode 87 - 12 June 2018 I Mazhav...,Mazhavil Manorama,24,2018-06-13T05:00:02.000Z,"mazhavil manorama|""bhramanam full episode""|""gt...",386319,1216,453,697,https://i.ytimg.com/vi/mV6aztP58f8/default.jpg,False,False,False,Subscribe to Mazhavil Manorama now for your da...,Entertainment
37350,qxqDNP1bDEw,18.14.06,Nua Bohu | Full Ep 285 | 13th June 2018 | Odia...,Tarang TV,24,2018-06-13T15:07:49.000Z,"tarang|""tarang tv""|""tarang tv online""|""tarang ...",130263,698,115,65,https://i.ytimg.com/vi/qxqDNP1bDEw/default.jpg,False,False,False,Nuabohu : Story of a rustic village girl who w...,Entertainment


In [15]:
#Creating a seperate functions for plotting as plotly has no way to update graph inside for loop
#def plot_(data):
   # fig = px.bar(data,x = s.index, y = s.values,title = i,labels = {'y':"Number of times trended"},width = 800,height =400 )
   # fig.show()
top_cat = []
top_channels_in_cat = {}
def top_channels_categoryWise(data,cat_names):
    for i in cat_names:
        cat_data = data[data['Category'] == i]
        cat_data = cat_data['channel_title'].value_counts().head(10).to_frame('Times_trended')
        if(cat_data['Times_trended'].sum() > 1000):
            top_cat.append(i)
        if(cat_data['Times_trended'].sum() > 0):
            top_channels_in_cat[i] = cat_data.index[0]
        #print(cat_data.head(5))
        fig2 = px.bar(cat_data,x = cat_data.index, y = 'Times_trended',title = i,labels = {'y':"Number of times trended"},width = 800,height =400 )
        fig2.show()
        
    

In [16]:
top_channels_categoryWise(videos_with_cat,cat_names)

# Conclusions from above Analysis
**Top Categoirs:**


In [17]:
for i in top_cat:
    print(f"{i}\n")

Entertainment

Comedy

News & Politics



**Top Channels in Every Category**

In [18]:
print('\033[1m' + "Category   Channel")
for i in top_channels_in_cat:
    print(f"{i}::{top_channels_in_cat[i]}")

Category   Channel
Travel & Events::TV9 Telugu
Movies::Rhythm Boyz
Entertainment::etvteluguindia
Education::Study IQ education
Comedy::ETV Plus India
People & Blogs::ETV Jabardasth
Shows::VikatanTV
Howto & Style::5-Minute Crafts
News & Politics::V6 News Telugu
Science & Technology::Technical Guruji
Gaming::Clash of Clans


In [19]:
#Checking the titles of the videos

In [20]:
videos_with_cat.head(2)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,Category
0,kzwfHumJyYc,17.14.11,Sharry Mann: Cute Munda ( Song Teaser) | Parmi...,Lokdhun Punjabi,1,2017-11-12T12:20:39.000Z,"sharry mann|""sharry mann new song""|""sharry man...",1096327,33966,798,882,https://i.ytimg.com/vi/kzwfHumJyYc/default.jpg,False,False,False,Presenting Sharry Mann latest Punjabi Song Cu...,Unknown
1,zUZ1z7FwLc8,17.14.11,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",HJ NEWS,25,2017-11-13T05:43:56.000Z,"पीरियड्स के समय|""पेट पर पति करता ऐसा""|""देखकर द...",590101,735,904,0,https://i.ytimg.com/vi/zUZ1z7FwLc8/default.jpg,True,False,False,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",News & Politics


In [21]:
videos_with_cat.index

RangeIndex(start=0, stop=37352, step=1)

In [22]:
def top_tags(data):
    data.reset_index()
    tag_in_cat = {}
    for ct_names in cat_names:
        tag_in_cat.clear()
        cat_data = data[data['Category'] == ct_names]
        for idx,row in cat_data.iterrows():
            tags = row['tags'].split("|")
            for tg in tags:
                if tg not in tag_in_cat:
                    tag_in_cat[tg] = 1
                else:
                    tag_in_cat[tg] = tag_in_cat[tg] + 1
        print(f"Top categories in {ct_names} = \n")
        tags  = Counter(tag_in_cat)
        print("For category %s"%ct_names)
        x = []
        y = []
        for k,v in tags.most_common(10):
            x.append(k)
            y.append(v)
        res_dict = {'Tag': x,'Count':y}
        res_dict = pd.DataFrame(res_dict,columns = ['Tag','Count'])
        fig = px.bar(res_dict, x = 'Tag',y = 'Count',width = 500,height = 200)
        fig.show()
       # print(res_dict)
        


# Top tags in every Category

In [23]:
top_tags(videos_with_cat)

Top categories in Thriller = 

For category Thriller


Top categories in Sci-Fi/Fantasy = 

For category Sci-Fi/Fantasy


Top categories in Shorts = 

For category Shorts


Top categories in Travel & Events = 

For category Travel & Events


Top categories in Drama = 

For category Drama


...

Top categories in Videoblogging = 

For category Videoblogging


Top categories in News & Politics = 

For category News & Politics


Top categories in Science & Technology = 

For category Science & Technology


Top categories in Short Movies = 

For category Short Movies


Top categories in Gaming = 

For category Gaming
